## Final Project Submission

Please fill out:
* Student name: Danyal Andriano
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 


In [ ]:
#!pip install folium
#!pip install geopy

In [1]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import stats
from scipy.stats import boxcox
from math import sqrt
import sqlite3
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import *
import statsmodels.stats.outliers_influence as st_inf
from statsmodels.stats.anova import anova_lm
import missingno as msno
import folium
from folium import plugins
from folium.plugins import HeatMap
import geopy.distance
from pandasql import sqldf
%matplotlib inline
#%matplotlib qt

In [2]:
#Import data 
df = pd.read_csv('kc_house_data.csv')
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


The data consists of 20 variable columns and an index. There are 21596 entries. Since n far exceeds p (the number of variables in the set) we should have no power issues.

- sqft_basement is float, but contains place holders for unknown data. Explore further below for decision to be made. 
- waterfront appears to be bool, but to have missing values; left for further exploration below
- view appears to be bool but to also have some missing values; left for further analysis below
- year data is interval; further analysis to determine how feature will be used for modeling
- zipcode and location data will be removed from the main dataset for now and returned to later... there are too many cities and zipcodes to use as a categorical set of predictors in the model. However, location data could be used to access other demographic data and zipcodes can be used grossly to divide the county into 4 segments (East, West, North, South) if time permits. 

## Data Clean-up

In [3]:
# Column headings stripped of white spaces and made lower case for easier processing
df.columns.str.lower().str.strip()

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

## Location Data

### QUESTION: How can the location data be best used to predict house price? 
- Does distance from the most desirable residential area predict house value?
- what can the zipcodes tell us about the average price for houses in an area?

In [4]:
df.lat.median()
df.long.median()

-122.23100000000001

In [5]:
df.zipcode.astype(int);
df.zipcode = df['zipcode'].astype(str);

In [6]:
zips_df = df.groupby(['zipcode']).mean()
zips_df.reset_index(level=0, inplace=True)
zips_df.price.min().round()

234284.0

In [7]:
zips_df

,zipcode,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,98001,4.651274e+09,2.811949e+05,3.393352,2.011773,1903.783934,14967.002770,1.430748,0.000000,0.094708,3.335180,7.296399,1723.756233,1980.867036,28.038732,47.309106,-122.270704,1830.099723,11218.875346
1,98002,4.850894e+09,2.342840e+05,3.326633,1.839196,1627.743719,7517.633166,1.334171,0.000000,0.010050,3.743719,6.693467,1522.537688,1967.773869,60.212121,47.308780,-122.213357,1479.030151,7585.482412
2,98003,4.674182e+09,2.941113e+05,3.357143,2.047321,1928.882143,10603.096429,1.310714,0.000000,0.214286,3.371429,7.542857,1663.550000,1976.885714,16.641667,47.315741,-122.310054,1876.196429,9746.178571
3,98004,4.394237e+09,1.356524e+06,3.854890,2.527603,2909.022082,13104.220820,1.432177,0.003534,0.306962,3.495268,8.687697,2419.842271,1971.470032,213.759542,47.616183,-122.205189,2674.700315,12805.022082
4,98005,4.961872e+09,8.102897e+05,3.851190,2.424107,2656.803571,19928.785714,1.279762,0.000000,0.095238,3.696429,8.488095,2160.196429,1969.744048,57.985507,47.611532,-122.167268,2567.863095,18367.773810
5,98006,4.621241e+09,8.599386e+05,3.841365,2.555723,2888.295181,13383.435743,1.459839,0.009132,0.631791,3.658635,8.795181,2300.383534,1978.720884,82.798054,47.558022,-122.146839,2785.612450,12159.959839
6,98007,4.450672e+09,6.172540e+05,3.815603,2.173759,2182.056738,10662.702128,1.411348,0.000000,0.028369,3.631206,7.964539,1885.673759,1971.070922,51.767241,47.610522,-122.142078,2124.822695,9579.900709
7,98008,4.594706e+09,6.456282e+05,3.734982,2.088339,2133.445230,10188.448763,1.181979,0.035156,0.477032,3.667845,7.653710,1713.088339,1966.325088,59.139241,47.613312,-122.115834,2021.130742,9494.996466
8,98010,3.879195e+09,4.236660e+05,3.290000,2.027500,2137.590000,55465.300000,1.480000,0.000000,0.100000,3.530000,7.400000,2000.340000,1979.180000,241.914634,47.333210,-121.999470,2008.760000,47830.270000
9,98011,4.284961e+09,4.903771e+05,3.548718,2.278205,2253.097436,11314.138462,1.502564,0.000000,0.061856,3.220513,7.774359,1954.558974,1982.148718,49.861635,47.755192,-122.201174,2248.076923,9512.087179


In [8]:
map_zips = folium.Map([47.608013, -122.335167], zoom_start=11)
map_zips;

In [9]:
# Create lists for folium markers

# pairs of latitude and longitude coordinates  
locations = zips_df[['lat', 'long']]
location_list = locations.values.tolist()
len(location_list) 

70

In [10]:
price = zips_df['zipcode'] + " $" + zips_df['price'].astype(int).astype(str)
price = price.values.tolist()
price[0:5]

['98001 $281194',
 '98002 $234284',
 '98003 $294111',
 '98004 $1356523',
 '98005 $810289']

In [11]:
folium.Marker([47.608013, -122.335167], popup= "Seattle").add_to(map_zips)
map_zips;

In [12]:
for i in range(0, len(location_list)):
    folium.Marker(location_list[i], popup=price[i]).add_to(map_zips)
map_zips


In [13]:
coord = df[['lat', 'long']]
coordinates = coord.values.tolist()
len(coordinates)

21597

In [14]:
distances = []
for i in range(len(coordinates)):
    coords_1 = (47.625840, -122.233540) # Bellevue    #(47.608013, -122.335167) Seattle
    coords_2 = coordinates[i]
    distances.append(geopy.distance.vincenty(coords_1, coords_2).miles)

df['distance'] = pd.DataFrame(distances)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,distance
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,7.995528
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,7.689150
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,7.741887
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,10.403648
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,8.828511


In [15]:
df.distance.describe()

count    21597.000000
mean        10.860158
std          6.196645
min          0.147549
25%          6.518003
50%          9.014470
75%         13.836673
max         43.284758
Name: distance, dtype: float64

In [16]:
df.price.corr(df.distance)

-0.3996844228649356

### Drop Columns

In [ ]:
#column_names = ['id', 'zipcode', 'lat', 'long']
location_df = df.iloc[:, 16:19]
location_df.head()

In [ ]:
# remove date - not time series analysis and we have the years as time reference better suited to an OLS regression. 
# remove id as the integer index appears to be a simpler and more effective identification
# remove location data for now
to_drop = ['id', 'date', 'zipcode']    #'lat', 'long']
df.drop(to_drop, axis=1, inplace=True)


### Placeholders and Missing Values

In [ ]:
# visual inspection of the numerical variables indicates no other placeholders (very unusual values). However, extreme values 
# will be further explored below. 
for col in df.columns:
    print(col)
    print(df[col].unique())

#### sqft_basement

In [ ]:
# As sqrt_basement should be a numerical data type, but is a string, I had reason to believe there may be a string value present.
# The data were visually inspected and a '?' placeholder is present in the data
(df['sqft_basement'] == '?').sum()

In [ ]:
# sqft_basement has 454 unknown values 
# First we can change the '?' placeholder to np.NaN

df['sqft_basement'].replace('?', np.NaN, inplace=True) 

In [ ]:
# change the variable into a float so that we can make use of decsriptive stats to deal with the unknown values
df['sqft_basement'] = df['sqft_basement'].astype(float)
df['sqft_basement'].describe()

In [ ]:
def value_finder(data, value):
    placeholder_lst = []
    for col in data.columns:
        placeholder_lst.append({'Column': col, value: len(df[df[col] == value].index)})
    return placeholder_lst

In [ ]:
# Based on the above statistics, the data were further investigated for values of 0.0
#%%time
value_finder(df, 0.0)

In [ ]:
# 2% of the data in sqft_basement is null. This is a small enough percentage where replacement via central measure should not impact 
# the distribution or analysis substantially
df.sqft_basement.isna().sum()/len(df.sqft_basement)*100

In [ ]:
# Inspection of sqft_basement reveals that the distribution is positively skewed. Therefore, in this case the median rather than 
# the mean can be used as a replacement value (i.e., the central measure least likely to change the shape of the distribution)
df['sqft_basement'].replace(np.NaN, 0.0, inplace=True) 
df.head()

In [ ]:
#df.plot.scatter(x='sqft_basement', y='price')
sns.regplot(x='sqft_basement', y='price', data =df)

In [ ]:
# despite many values of zero, sqft_basement was left in the analysis. 
print('Pearson correlation coefficent between sqft basement and price = {}'.format(round(df.sqft_basement.corr(df.price), 2)))

#### MIssing Data

In [ ]:
msno.matrix(df)
#plt.autoscale(False)
#sns.heatmap(df.isnull(), cbar=False);


In [ ]:
df.isna().sum()

#### WATERFRONT

In [ ]:
# Waterfront indicates whether a house has a view to the waterfront. 
df.waterfront.unique()
df.waterfront.value_counts()

In [ ]:
#Null values were temporarily dropped to explore further
df.waterfront.dropna(axis=0, inplace=True);
df.waterfront.isna().sum()

In [ ]:
waterfront_ave = round(df.groupby(['waterfront']).mean(), 2)
waterfront_ave.head()

In [ ]:
# replace waterfront missing values with NaN category

df.waterfront = df.waterfront.astype(str)

df.waterfront.replace(np.NaN, 'NaN', inplace=True)
df.waterfront.replace(0.0, 'False', inplace=True)
df.waterfront.replace(1.0, 'True', inplace=True)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, figsize=(9,9))
sns.boxplot(x="waterfront", y="price", data=df, ax=ax1)
sns.boxplot(x="waterfront", y="sqft_living", data=df, ax=ax2)
sns.boxplot(x="waterfront", y="grade", data=df, ax=ax3)
f.tight_layout()

In [ ]:
# waterfront contains only a very small percentage of houses; however, there appears to be a real difference in size and price,
# with the waterfront = 1 median being within outlier range for price and beyond the 75th percentile for sqft_living. 
# These houses are perhaps best left out of the analysis altogether, as they may represent a different population of mansions and 
# wealthier properties. 

In [ ]:
percentage_waterfront_view = 146/len(df.waterfront)*100
print(round(percentage_waterfront_view,2))

Houses with a waterfront view made up 0.7% of the data, but clearly had median values out of range for those properties without
a view (i.e., we could argue that they come from a significantly different population). Visual inspection indicates that the 
category of missing values is more similar to the non-view category within waterfront and will therefore be treated a such. Houses with a waterfront view were therefore removed from the dataset and the column was dropped. 

In [ ]:
df = df[df['waterfront'] != '1.0']
#df.where(df['waterfront'] != 1.0, inplace=True)
#df.loc[df['waterfront'] != 1.0]

In [ ]:
# check that the rows were indeed dropped
df.waterfront.unique()

In [ ]:
# remove the waterfront column
df.drop('waterfront', axis=1, inplace=True);

In [ ]:
df.head()

In [ ]:
df.price.corr(df.distance)

#### VIEW

In [ ]:
df.view.isna().sum()

In [ ]:
df.view.value_counts()

In [ ]:
#temp_explore = df.dropna(subset=['view']).plot(x='view', y='price', kind='scatter')
df.plot(x='view', y='price', kind='scatter')

In [ ]:
print('Pearson correlation coefficent between view and price = {}'.format(df.view.corr(df.grade)))

View is a measure of whether or not the property has been viewed. The value counts and scatter plot show that it is clearly interval data, with 0 being no views and the maximum number of views being 4. The vast majority of properties have not been viewed, which is resulting in an extreme possitive skew. There are also 62 missing values. Furthermore, when viewed in relation to the target variable, there appears to be no noteworthy relationship between between these variables. This is confirmed by the modest correlation between the two, which is likely being increased by the few apparent outliers. View will therefore be dropped from the data set. 

In [ ]:
df.drop('view', axis=1, inplace=True)

In [ ]:
df.head()

#### YEAR RENOVATED

In [ ]:
df.yr_renovated.isna().sum()

In [ ]:
df.hist('yr_renovated')

In [ ]:
df_temp = df.loc[df['yr_renovated'] > 0.0]
len(df_temp)/len(df)*100
#f_temp.plot.scatter('yr_renovated', 'price');

In [ ]:
df_temp.hist('yr_renovated')

In [ ]:
print('Pearson correlation coefficent between price and year renovated = {}'.format(round(df_temp.price.corr(df.yr_renovated), 2)))

Due to the very small percentage of houses with the year of renovations recorded (3%) this variable will be dropped. It is clear that houses that were renovated have been given a value of 0, indicating that in addition to all of the missing data for this column, a value of 0 also indicates missing data. 

In [ ]:
df.drop('yr_renovated', axis=1, inplace=True)

In [ ]:
df.head()

### Data Type Corrections

- price: target variable (float)
- bedrooms: number of bedrooms (int)
- bathrooms: number of bathrooms (float)
- sqft_living: square footage of the home (int)
- sqft_lot: square footage of the lot (int)
- floors: Total floors (levels) in house (int)
- grade: overall grade given to the housing unit, based on King County grading system (int)
- sqft_above: square footage of house apart from basement (int)
- sqft_basement: square footage of the basement (float)
- yr_built: Built year (years)
- sqft_living15: Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area
- sqft_lot15: lotSize area in 2015(implies-- some renovations 


In [ ]:
df.sqft_basement = df.sqft_basement.astype(int)


In [ ]:
df.info()

In [ ]:
df.head()

### Feature Engineering and Multicollinearity

In [ ]:
# Year built was replaced with a variable called age
df['age'] = 2018 - df['yr_built']

In [ ]:
df.age.hist()

In [ ]:
df.drop('yr_built', axis=1, inplace=True)


In [ ]:
df.head()

In [ ]:
df['renovated_living'] = df['sqft_living'] - df['sqft_living15']
df['renovated_lot'] = df['sqft_lot'] - df['sqft_lot15']


In [ ]:
# Two new variables were created for the size of the renovations made to the living and lot. 
df.renovated_living.hist()
print(df.renovated_living.describe().round())

In [ ]:
df.renovated_lot.describe().round()

### Removing extreme values

In [ ]:
normalized_df=(df-df.mean())/df.std()
len(normalized_df)

In [ ]:
merged = df.merge(normalized_df, suffixes=('', '_z'), left_index=True, right_index=True)
merged.columns

In [ ]:
#len(merged.loc[merged['price_z'] > 2.5])/len(merged)  #
tails = len((merged.loc[merged['price_z'] > 3]) | (merged.loc[merged['price_z'] < - 3]))/len(merged)        
tails*100

In [ ]:
merged.loc[merged['price_z'] > 10]

In [ ]:
merged.loc[merged['price'] > 3999999]

Houses over 4 million were shown to have extremely high z scores (> 10) and were excluded. Usually values over 3-4 STD can be considered outliers, but price is not normally distributed, so extreme cases were chosen for the time being. Further outlier analysis will be considered for other extreme cases. 

In [ ]:
df2 = df.loc[df['price'] < 3999999]

In [ ]:
df2.head()
len(df2)

In [ ]:
df3 = df2

In [ ]:
df3.head()

In [ ]:
# Keep track of total number of subplots
k = len(df.columns)
# n will be the number of chart columns. Change this to suit individual needs.  
# m will be the calculated number of required rows based on k and n
n = 2
m = (k - 1) // n + 1
#Create a figure and array of axes with required number of rows and columns.
fig, axes = plt.subplots(m, n, figsize=(12,12))
#Iterate through columns, tracking the column name and which number we are at i. Within each iteration, plot.

for i, (name, col) in enumerate(df.iteritems()):
    r, c = i // n, i % n
    ax = axes[r, c]
    col.hist(ax=ax)
    ax2 = col.plot.kde(ax=ax, secondary_y=True, title=name)
    ax2.set_ylim(0)

fig.tight_layout()

In [ ]:
df2.lat.hist()

In [ ]:
data_norm = pd.DataFrame([])
data_norm["logprice"] = np.log(df2["price"])
data_norm['sqrtage'] = np.sqrt(df2['age'])
data_norm['logsqft_living'] = np.sqrt(df2['sqft_living'])
data_norm['logdist'] = np.log(df2['distance'] +1)
data_norm['logsqft_lot15'] = np.log(df2['sqft_lot15'] +1)
data_norm['logsqft_living15'] = np.log(df2['sqft_living15'])
data_norm.hist(figsize  = [10,6]);

In [ ]:
df3 = df3.merge(data_norm, left_index=True, right_index=True)


In [ ]:
df3.head()

In [ ]:
to_drop = ['price', 'sqft_living', 'distance','sqft_living15', 'sqft_lot15']
df3.drop(to_drop, axis=1, inplace=True)

In [ ]:
# Set the style of the visualization
sns.set(style="white")

# Create a covariance matrix
corr = df3.corr()

# Generate a mask the size of our covariance matrix
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16,13))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220,10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8, center=0, square=True, linewidths=.4, annot=True, cbar_kws={'shrink':0.6})

- Age was found to have no correlation with price. Interestingly, though, it did have significant inverse correlations with a number of other variables that did correlate with price (for instance, bathrooms, floors, grade), and so it may have an indirect relationship with price.
- The size of the lot did not seem related to price, in all instances (original lot size, recent lot size and renovations to lot). Therefore, all lot variables will be dropped.
- Condition showed no relationship with price and will be dropped

In [ ]:
#sns.scatterplot(x='grade', y='age', data=df)

In [ ]:
to_drop = ['renovated_lot', 'sqft_lot15', 'condition','sqft_lot']
df2.drop(to_drop, axis=1, inplace=True)


In [ ]:
df.describe()

In [ ]:
# drop negative values for boxcox transformation
#df.drop('renovated_living', axis=1, inplace=True)
#df.drop('long', axis=1, inplace=True)

In [ ]:
df2.describe()

In [ ]:
# Set the style of the visualization
sns.set(style="white")

# Create a covariance matrix
corr = df2.corr()

# Generate a mask the size of our covariance matrix
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(13,10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220,10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8, center=0, square=True, linewidths=.5, annot=True, fmt=".2f", cbar_kws={'shrink':0.5})

### Feature Selection and Model Building

### Hierarchical Selection (based on interpretation)

Interpretation was based on visual inspection of the correlation matrix and understanding of the variables
- Many of the variables are measures of the size of the property and intercorrelated; therefore, I determined beforehand that only some of these variables are accounting uniquely for variance in price. sqft_living had the highest correlation with price and from this was most likely to be significantly related to price. 
- Distance, lat and long all had moderate relationships with price
- Grade also a strong correlation with price
- Age had no relationship to price; however, it did have moderate inverse relationships with certain variables that were correlated with price, which is an indication that it may be a moderator. 

##### Selection of size features 

- Model 1:   price ~ sqft_living                          = 49.1% 
- Model 2.1: price ~ sqft_living + grade                  = 54.0% (R-square change: 0.049)
- Model 2.2: price ~ sqft_living + age                    = 52.3% (R-square change: 0.023)
- Model 3:   price ~ sqft_living + grade + age            = 61.2% (R-square change: 0.121)
- Model 4:   price ~ sqft_living + grade + age + distance = 68.9% (R-square change: 0.077)

All other variables failed to account for more than 1% variance and in some introduced error and dimensionality into the model, resulting in a lower R-squared. 

In [ ]:
# all other variables accounted for less variance when added into the model
formula1 = "logprice ~ grade + logdist + logsqft_living +sqft_lot+ lat + condition"                     #logsqft_living  
model1 = ols(formula= formula1, data=df3_z).fit()
model1.summary()

In [ ]:
formula2 = "logprice ~ logsqft_living+grade"   
model2 = ols(formula= formula2, data=df3_z).fit()
model2.summary()

In [ ]:
formula3 = "logprice ~ logsqft_living+sqrtage"    
model3 = ols(formula= formula3, data=df3_z).fit()
model3.summary()

In [ ]:
formula4 = "logprice ~ logsqft_living+grade+sqrtage"   
model4 = ols(formula= formula4, data=df3_z).fit()
model4.summary()

In [ ]:
formula5 = "logprice ~ logsqft_living+grade+sqrtage+logdist"   
model5 = ols(formula= formula5, data=df3_z).fit()
model5.summary()

In [ ]:
#bathrooms+bedrooms+sqft_basement+floors+sqft_above+sqft_living15"
formula6 = "logprice ~ logsqft_living+grade+sqrtage+logdist+lat+bathrooms+bedrooms+sqft_basement+floors+sqft_above+logsqft_living15"       
model6 = ols(formula= formula6, data=df3_z).fit()
model6.summary()

In [ ]:
anovaResults = anova_lm(model1, model2, model3, model4, model5, model6)
anovaResults
#np.round()

##### Selection of location features

In [ ]:
formula2_1 = "logprice ~ logsqft_living+grade+sqrtage+logdist"   
model2_1 = ols(formula= formula2_1, data=df3_z).fit()
model2_1.summary()

In [ ]:
formula2_2 = "logprice ~ logsqft_living+grade+sqrtage+logdist+lat"   
model2_2 = ols(formula= formula2_2, data=df3_z).fit()
model2_2.summary()

In [ ]:
formula2_3 = "price ~ sqft_living+grade+age+lat+long+distance"   
model2_3 = ols(formula= formula2_3, data=df2).fit()
model2_3.summary()

Model2.1: price ~ sqft_living+grade+age+distance = 68.9%
Model2.2: price ~ sqft_living+grade+age+lat+long = 65.8%
Model2.3: price ~ sqft_living+grade+age+lat+long+distance = 68.9%

In [ ]:
anova_location_data = anova_lm(model2_1, model2_2)
anova_location_data

### Recursive Feature Selection

In [ ]:
king_features = df3.drop('logprice', axis=1);

In [ ]:
king_features.columns;

In [ ]:
df2.head();

In [ ]:
target = df3['logprice'];

In [ ]:
x = king_features
y = target

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 14)
selector = selector.fit(x, y)

In [ ]:
selector.support_ 

In [ ]:
selected_columns = x.columns[selector.support_ ]
linreg.fit(x[selected_columns],y)

In [ ]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
yhat = linreg.predict(x[selected_columns])

In [ ]:
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1)

In [ ]:
print(SS_Residual)
print(SS_Total)
print(r_squared)
print(adjusted_r_squared)

In [ ]:
df3_z = (df3-df3.mean())/df3.std()
df3_z.head()

In [ ]:
xz = df3_z.drop('logprice', axis=1)
yz = df3_z.logprice

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = np.mean(cross_val_score(linreg, xz, yz, cv=5, scoring="neg_mean_squared_error"))
cv_10_results = np.mean(cross_val_score(linreg, xz, yz, cv=10, scoring="neg_mean_squared_error"))
cv_20_results = np.mean(cross_val_score(linreg, xz, yz, cv=20, scoring="neg_mean_squared_error"))

print(1 - (- cv_5_results))
print(1 - (- cv_10_results))
print(1 - (- cv_20_results))

In [ ]:
mod1 = linreg.fit(xz,yz)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
linreg.coef_

In [ ]:
yhat1 = linreg.predict(xz)

In [ ]:
SS_Residual = np.sum((yz-yhat1)**2)
SS_Total = np.sum((yz-np.mean(yz))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(yz)-1)/(len(yz)-xz.shape[1]-1)

In [ ]:
print(SS_Residual)
print(SS_Total)
print(r_squared)
print(adjusted_r_squared)

In [ ]:
X_int = sm.add_constant(xz)
mod2 = sm.OLS(yz,X_int).fit()
mod2.summary()

### Stepwise Selection

In [ ]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
result = stepwise_selection(xz, yz, verbose = True)
print('resulting features:')
print(result)

### Forwards Selection

In [ ]:
import statsmodels.formula.api as smf

In [ ]:

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [ ]:
model = forward_selected(df3_z, 'logprice')

print(model.model.formula)
# sl ~ rk + yr + 1

print(model.rsquared_adj)

### Transformations and Diagnostics

In [ ]:
df2_final = df2.loc[:,['price', 'sqft_living', 'grade', 'age', 'distance']]

In [ ]:
x = df2_final.drop('price', axis=1)

In [ ]:
y = df2.price

In [ ]:
linreg = LinearRegression()
linreg.fit(x,y)

In [ ]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
yhat = linreg.predict(x)

In [ ]:
residuals = y - yhat
#residuals.hist()
sns.distplot(residuals)
plt.xlim(-1000000, 1000000)

In [ ]:
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1)

In [ ]:
print(SS_Residual)
print(SS_Total)
print(r_squared)
print(adjusted_r_squared)

In [ ]:
final_formula = "price ~ sqft_living+grade+age+distance"   
fmodel = ols(formula= final_formula, data=df2_final).fit()

In [ ]:
#sns.set(style="ticks", color_codes=True)
#sns.pairplot(df_final, kind='reg', diag_kind='kde', plot_kws={'line_kws':{'color': 'red', 'linewidth':1}});

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.influence_plot(fmodel, ax=ax, criterion="cooks")

In [ ]:
m = fmodel
infl = m.get_influence()
sm_fr = infl.summary_frame()
sm_fr.head()


In [ ]:
sm_fr.loc[(sm_fr['standard_resid'] > 3) |  (sm_fr['standard_resid'] < -3)]  

In [ ]:
328/len(df2_final)*100

In [ ]:
len(df2_final)

In [ ]:
merged2 = df2_final.merge(sm_fr, suffixes=('', '_z'), left_index=True, right_index=True)
len(merged2)

In [ ]:
merged2.loc[(merged2['standard_resid'] > 3) |  (merged2['standard_resid'] < -3)] 

In [ ]:
final = merged2.drop(merged2[(merged2.standard_resid < -3) | (merged2.standard_resid > 3)].index)

In [ ]:
len(final)

In [ ]:
final_formula = "price ~ sqft_living+grade+age+distance"   
fmodel2 = ols(formula= final_formula, data=final).fit()
fmodel2.summary()

In [ ]:
to_drop = ['dfb_Intercept', 'dfb_sqft_living', 'dfb_grade', 'dfb_age', 'dfb_distance', 'cooks_d', 'standard_resid', 'hat_diag', 'dffits_internal', 'student_resid', 'dffits']
final.drop(to_drop, axis=1, inplace=True)

In [ ]:
final.head()

In [ ]:
# min-max scaling susceptible to outliers, use z scores
scaled = (df2_final-df2_final.mean())/df2_final.std()
len(scaled)

In [ ]:
scaled.head()

In [ ]:
scaled2 = (final-final.mean())/final.std()
len(scaled2)

In [ ]:
x1 = scaled2.drop('price', axis=1)
y1 = scaled2.price

In [ ]:
cv_5_results = np.mean(cross_val_score(linreg, x1, y1, cv=5, scoring="neg_mean_squared_error"))
cv_10_results = np.mean(cross_val_score(linreg, x1, y1, cv=10, scoring="neg_mean_squared_error"))
cv_20_results = np.mean(cross_val_score(linreg, x1, y1, cv=20, scoring="neg_mean_squared_error"))

print(1 - (- cv_5_results))
print(1 - (- cv_10_results))
print(1 - (- cv_20_results))

In [ ]:
x2 = final.drop('price', axis=1)
y2 = final.price

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = np.mean(cross_val_score(linreg, x2, y2, cv=5, scoring="neg_mean_squared_error"))
cv_10_results = np.mean(cross_val_score(linreg, x2, y2, cv=10, scoring="neg_mean_squared_error"))
cv_20_results = np.mean(cross_val_score(linreg, x2, y2, cv=20, scoring="neg_mean_squared_error"))

print(cv_5_results)
print(cv_10_results)
print(cv_20_results)

In [ ]:
x3 = scaled.drop('price', axis=1)
y3 = scaled.price

In [ ]:
cv_5_results = np.mean(cross_val_score(linreg, x3, y3, cv=5, scoring="neg_mean_squared_error"))
cv_10_results = np.mean(cross_val_score(linreg, x3, y3, cv=10, scoring="neg_mean_squared_error"))
cv_20_results = np.mean(cross_val_score(linreg, x3, y3, cv=20, scoring="neg_mean_squared_error"))

print(1 - (- cv_5_results))
print(1 - (- cv_10_results))
print(1 - (- cv_20_results))

### EXTRA

In [ ]:
#df_num[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [ ]:
#df_num.price.value_counts(normalize=True) < 0.01;

In [ ]:
def placeholder1(data, placeholder):
    placeholder_lst = []
    for col in data.columns:
        placeholder_lst.append({'Column': col, placeholder: len(df[df[col] == placeholder].index)})
    return placeholder_lst

In [ ]:
df.plot.scatter('price', 'sqft_living')
#plt.colorbar()
#plt.xlim()
#sns.scatterplot(x = "price", y = "sqft_living", data=df)

BUILDING GRADE
    	Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc. 

#### Further analysis:
- location based spatial analysis
- modeling missing data for better replacement


In [ ]:
def map_points(df, lat_col='lat', lon_col='long', zoom_start=11, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lat_col: Column containing latitude (string)
        lon_col: Column containing longitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()

    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=pt_radius,
                                popup=row['zipcode'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        stations = df[cols_to_pull].as_matrix()
        curr_map.add_children(plugins.HeatMap(stations, radius=heat_map_radius))

    return curr_map

In [ ]:
map_points(df, plot_points=True, draw_heatmap=True, heat_map_weights_normalize=True,
          heat_map_weights_col='price')

In [ ]:
#outcome = 'price'
#predictors = df.drop('price', axis=1)
#pred_sum = "+".join(predictors.columns)
#formula = outcome + "~" + pred_sum